In [100]:
import pickle
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [77]:
# read in data from csv file to pandas dataframe. 
df = pd.read_csv('history_station.csv', keep_default_na=True, delimiter=',', skipinitialspace=True, encoding='Windows-1252')
df_weather = pd.read_csv('weather_data.csv', keep_default_na=True, delimiter=',', skipinitialspace=True, encoding='Windows-1252')

## Clean and prepare data in history_station

In [78]:
df.shape

(113337, 6)

In [79]:
df.head(5)

,id,number,update_date,available_bikes,available_bike_stands,weather
0,1,42,1680129628000,18,12,Clouds
1,2,42,1680129628000,18,12,Clouds
2,3,42,1680134426000,18,12,Rain
3,4,30,1680134433000,14,6,Rain
4,5,54,1680134427000,2,31,Rain


In [80]:
# drop the first 2 rows (they are test data for py file)
df = df.drop(index=range(2))
df.head(5)

,id,number,update_date,available_bikes,available_bike_stands,weather
2,3,42,1680134426000,18,12,Rain
3,4,30,1680134433000,14,6,Rain
4,5,54,1680134427000,2,31,Rain
5,6,108,1680134531000,20,15,Rain
6,7,20,1680134425000,0,30,Rain


In [82]:
# convert timestamp to datetime
df['update_date'] = pd.to_datetime(df['update_date'], unit='ms')

# extract year, month, day, weekday and hour from datetime and create new columns
df['year'] = df['update_date'].dt.year
df['month'] = df['update_date'].dt.month
df['day'] = df['update_date'].dt.day
df['hour'] = df['update_date'].dt.hour
df['weekday'] = df['update_date'].dt.weekday

# create new column with formatted datetime
df['formatted_date'] = df['update_date'].apply(lambda x: x.strftime('%Y-%m-%d-%H'))

df.tail(5)

,id,number,update_date,available_bikes,available_bike_stands,weather,year,month,day,hour,weekday,formatted_date
113332,113333,39,2023-04-05 21:40:50,9,11,Clouds,2023,4,5,21,2,2023-04-05-21
113333,113334,83,2023-04-05 21:50:39,21,19,Clouds,2023,4,5,21,2,2023-04-05-21
113334,113335,92,2023-04-05 21:42:53,40,0,Clouds,2023,4,5,21,2,2023-04-05-21
113335,113336,21,2023-04-05 21:46:01,12,18,Clouds,2023,4,5,21,2,2023-04-05-21
113336,113337,88,2023-04-05 21:40:53,15,15,Clouds,2023,4,5,21,2,2023-04-05-21


In [83]:
# extract need coloumns
df = df[['number', 'available_bikes', 'weather', 'year', 'month', 'day', 'hour', 'weekday']]

df.head(5)

,number,available_bikes,weather,year,month,day,hour,weekday
2,42,18,Rain,2023,3,30,0,3
3,30,14,Rain,2023,3,30,0,3
4,54,2,Rain,2023,3,30,0,3
5,108,20,Rain,2023,3,30,0,3
6,20,0,Rain,2023,3,30,0,3


In [84]:
# group by date and hour and (station's)number
df['number'] = df['number'].astype(str)
df = df.groupby([df['month'], df['day'], df['hour'], 'number']).mean()
df = df.reset_index()

df

,month,day,hour,number,available_bikes,year,weekday
0,3,29,23,102,25.000000,2023.0,2.0
1,3,29,23,105,19.000000,2023.0,2.0
2,3,29,23,23,27.000000,2023.0,2.0
3,3,29,23,24,12.000000,2023.0,2.0
4,3,29,23,38,11.000000,2023.0,2.0
...,...,...,...,...,...,...,...
18882,4,5,21,95,11.833333,2023.0,2.0
18883,4,5,21,96,11.000000,2023.0,2.0
18884,4,5,21,97,24.000000,2023.0,2.0
18885,4,5,21,98,1.800000,2023.0,2.0


## Clean and prerare weather data

In [85]:
df_weather.shape

(1017, 4)

In [86]:
df_weather.head(5)

,time,temp,humidity,wind_speed
0,1680131894947,285.00,79.0,3.09
1,1680131908056,285.00,79.0,3.09
2,1680132324040,284.91,81.0,5.14
3,1680132924288,284.51,82.0,5.14
4,1680133524447,284.51,82.0,5.14


In [87]:
# convert timestamp to datetime
df_weather['time'] = pd.to_datetime(df_weather['time'], unit='ms')
# extract year, month, day, weekday and hour from datetime and create new columns
df_weather['year'] = df_weather['time'].dt.year
df_weather['month'] = df_weather['time'].dt.month
df_weather['day'] = df_weather['time'].dt.day
df_weather['hour'] = df_weather['time'].dt.hour
df_weather['weekday'] = df_weather['time'].dt.day_name()

# create new column with formatted datetime
df_weather['formatted_date'] = df_weather['time'].apply(lambda x: x.strftime('%Y-%m-%d-%H'))


In [88]:
df_weather.head(15)

,time,temp,humidity,wind_speed,year,month,day,hour,weekday,formatted_date
0,2023-03-29 23:18:14.947,285.00,79.0,3.09,2023,3,29,23,Wednesday,2023-03-29-23
1,2023-03-29 23:18:28.056,285.00,79.0,3.09,2023,3,29,23,Wednesday,2023-03-29-23
2,2023-03-29 23:25:24.040,284.91,81.0,5.14,2023,3,29,23,Wednesday,2023-03-29-23
3,2023-03-29 23:35:24.288,284.51,82.0,5.14,2023,3,29,23,Wednesday,2023-03-29-23
4,2023-03-29 23:45:24.447,284.51,82.0,5.14,2023,3,29,23,Wednesday,2023-03-29-23
5,2023-03-29 23:55:24.588,284.31,85.0,5.14,2023,3,29,23,Wednesday,2023-03-29-23
6,2023-03-30 00:05:24.775,284.31,85.0,5.14,2023,3,30,0,Thursday,2023-03-30-00
7,2023-03-30 00:15:24.935,283.86,86.0,2.68,2023,3,30,0,Thursday,2023-03-30-00
8,2023-03-30 00:25:25.138,283.69,89.0,5.14,2023,3,30,0,Thursday,2023-03-30-00
9,2023-03-30 00:35:25.247,283.58,89.0,5.14,2023,3,30,0,Thursday,2023-03-30-00


In [89]:
# extract needed columns
df_weather = df_weather[['temp','wind_speed','month','day','hour','weekday']]

In [90]:
# group by date and hour
df_weather = df_weather.groupby(['month', 'day', 'hour']).mean()

In [91]:
df_weather

temp  wind_speed
month day hour                        
3     29  23    284.706667    4.456667
      30  0     283.766667    4.730000
          1     283.578333    5.143333
          2     283.271667    6.158333
          3     282.723333    5.486667
...                    ...         ...
4     5   18    285.675000    4.973333
          19    285.086667    3.773333
          20    284.541667    3.430000
          21    283.970000    4.373333
          22    283.950000    2.907500

[168 rows x 2 columns]

## Merged the two data set

In [92]:
# merge weather data and station data
df_merged = pd.merge(df, df_weather, on=['month', 'day', 'hour'], how='left')

In [93]:
df_merged

,month,day,hour,number,available_bikes,year,weekday,temp,wind_speed
0,3,29,23,102,25.000000,2023.0,2.0,284.706667,4.456667
1,3,29,23,105,19.000000,2023.0,2.0,284.706667,4.456667
2,3,29,23,23,27.000000,2023.0,2.0,284.706667,4.456667
3,3,29,23,24,12.000000,2023.0,2.0,284.706667,4.456667
4,3,29,23,38,11.000000,2023.0,2.0,284.706667,4.456667
...,...,...,...,...,...,...,...,...,...
18882,4,5,21,95,11.833333,2023.0,2.0,283.970000,4.373333
18883,4,5,21,96,11.000000,2023.0,2.0,283.970000,4.373333
18884,4,5,21,97,24.000000,2023.0,2.0,283.970000,4.373333
18885,4,5,21,98,1.800000,2023.0,2.0,283.970000,4.373333


In [94]:
# divided merged data into different groups by their number

# get all unique values of number
numbers = df_merged['number'].unique()

# create a dictionary to store different data frames
df_dict = {}

# iterate each number in dict
for number in numbers:
    # get needed data
    mask = (df_merged['number'] == number)
    filtered_df = df_merged.loc[mask]
    
    # store
    df_dict[number] = filtered_df

df_dict['1']

,month,day,hour,number,available_bikes,year,weekday,temp,wind_speed
11,3,30,0,1,0.000000,2023.0,3.0,283.766667,4.730000
125,3,30,1,1,0.000000,2023.0,3.0,283.578333,5.143333
239,3,30,2,1,0.000000,2023.0,3.0,283.271667,6.158333
353,3,30,3,1,0.000000,2023.0,3.0,282.723333,5.486667
467,3,30,4,1,0.000000,2023.0,3.0,282.350000,5.828333
...,...,...,...,...,...,...,...,...,...
18317,4,5,17,1,6.833333,2023.0,2.0,286.023333,5.141667
18431,4,5,18,1,0.666667,2023.0,2.0,285.675000,4.973333
18545,4,5,19,1,1.333333,2023.0,2.0,285.086667,3.773333
18659,4,5,20,1,0.000000,2023.0,2.0,284.541667,3.430000


**temp,wind_speed,hour --> available bikes**

create a random forest regressor model

In [113]:
# create model for each station
for number in numbers:
    # dividing features and results
    X = df_dict[number][['hour','temp','wind_speed','weekday']]
    y = df_dict[number]['available_bikes']
    
    # Dividing the data set into a training set and a test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # create a random forest regressor with 100 trees
    rf = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
    
    # fit the model to the data
    rf.fit(X_train, y_train)
    
    # Model - serialising
    modelname = 'model' + number + '.pkl'
    with open(modelname, 'wb') as handle:
        pickle.dump(rf, handle, pickle.HIGHEST_PROTOCOL)